# Orchestrating a test lab using Jupyter Notebooks

## Getting started

The simplest possible way to automate deploying an experiment in Chameleon is to use a Jupyter notebook (like this one). Main reason is that it is a document/manual which you can run (shell) commands from and see the output. This document goes in more depth than [Getting Started Guide][1], namely it shows how to
1. Create a lease
1. Create ssh keypair
1. Get a floating IP
2. Create the required baremetal/virtual servers
1. Access both baremetal server
1. Build the test environment in the server. In this case we will run network traffic between the two servers.
1. Run experiment to our hearts content
1. Display results
1. Destroy the servers
1. Release floating IP
1. Delete SSH keypair
1. Destroy the lease.

from the command line.

### Limitations

1. We can only run non-interactive commands here. Think of this as a way to run scripts inside a document.

### Notes

1. Some of the resources will be associated with the user who is running this doc, in this case `$USER`.
1. If you are only a member of one project, you can skip this step, as the project will be selected for you by default. Otherwise, take a look at the current value of the environmental variable `OS_PROJECT_NAME`:

[1]: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html

In [ ]:
echo $OS_PROJECT_NAME

to ensure it matches the name of the project you want to run this lab on. If it does not, change it (uncommend the `export OS_PROJECT_NAME` line and set `your-project` to match the project name).

**NOTE:** If project has a nickname, you **must** use it instead.

In [ ]:
# Set up user's project (Replace 'your-project' with your project name)
# export OS_PROJECT_NAME='your-project'
echo "New project name = $OS_PROJECT_NAME"

**NOTE:** Just to be on the safe side, let's test it (we will explain the command later on) by asking for the list of ssh keypairs:

In [ ]:
openstack keypair list

If the output looks like this:
```
+------------+-------------------------------------------------+
| Name       | Fingerprint                                     |
+------------+-------------------------------------------------+
| defaultkey | d0:89:5b:61:6a:64:dd:c8:db:67:32:32:45:71:b0:b8 |
+------------+-------------------------------------------------+
```
you can continue to the next step. However, if it looks something like this:
```
The request you have made requires authentication. (HTTP 401) (Request-ID: req-76ad404f-0043-45e9-84cf-0504843888ab)
```
figure out what is going on before continuing. One thing to check is whether the project name you provided works (the nickname issue mentioned before is one possible reason).

You can also set the site you want to use via the `OS_REGION_NAME` setting; this defaults to `CHI@UC`.

In [ ]:
# Set region (Optional, default to 'CHI@UC')
# export OS_REGION_NAME='CHI@UC'
echo $OS_REGION_NAME

### Define some variables
We should define the names for our
  1. Lease
  1. Servers that belong to this lease. In this example we will call them `$SERVER_NAME-left` and `$SERVER_NAME-right` due to lack of imagination. 
  1. Private network
  1. Public network
  1. SSH key. By default Jupyter places the user's ssh keys in `~/work/.ssh`, but that should not stop us from placing them wherever we need them. Just to be different, we will put it in `~/.ssh`. Also, for this example, we delete the keys as part of the tearing down procedure. Note that `~/.ssh` is destroyed whenever the jupyter instance is deleted while `~/work/.ssh` remains
  1. Type for the instance/node/servers
  1. How many nodes we will need. Always get a least an extra one since one of the servers we want to build might be assigned to a flakey physical host. If we have spares, openstack will simply drop it and go to the next one.

in advance so we do not run the risk of mistyping them running a command. Here is ASCII art showing the layout:
  ```bash
  
        +-------------------+
        | Jupyter notebook  | (we are here)
        | server            | 
        +-------------------+
                |
                | (public network)
     <=><       |
                |        ><=>
                |
                | <--- Witchcraft happens here
   .................................
   :            | (private network) :
   :   +-------------------+        :
   :   | SERVER_NAME-left  |        :
   :   +-------------------+        :
   :           |                    :
   :   +-------------------+        :
   :   | SERVER_NAME-right |        :
   :   +-------------------+        :
   .................................
   
  ```


In [ ]:
export LEASE_NAME="$USER-test"
export SERVER_NAME="$USER-server"
export PRIVATE_NETWORK_NAME="sharednet1"
export PUBLIC_NETWORK_NAME="public"
export SSHKEY_FILE="$HOME/work/.ssh/work/$USER-chameleon"
export SSHKEY_NAME="ChameleonKey"
export NODE_TYPE="compute_haswell"
export NUM_SERVERS=2

These environmental variables only exist within the scope of this Jupyter document.

### Create a lease 
Specifically, we will create the lease `$LEASE_NAME`. 

In [ ]:
blazar lease-create --physical-reservation \
   min="$NUM_SERVERS",max=$((NUM_SERVERS + 1 )),resource_properties='["=", "$node_type", "'"$NODE_TYPE"'"]' "$LEASE_NAME"

Before we continue let's verify if the lease was successful created. This might take a few minutes, or just crash horribly. Since we are automating this, we need to account for these options.

In [ ]:
lease_status=""

# Lease in a sorry state
while [[ $lease_status == "TERMIN"* ]] || [[ $lease_status == "ERROR" ]] 
do
   echo "Lease it is in a sorry state. Restarting it."
   # Delete old lease
   blazar lease-delete "$LEASE_NAME"
   blazar lease-create --physical-reservation \
      min=1,"max=$MAX_SERVERS",resource_properties='["=", "$node_type", "$NODE_TYPE"]' "$LEASE_NAME"
   lease_status=$(blazar lease-show --format value -c status "$LEASE_NAME")
done
echo "Lease creation successfuly started."

# Now wait for lease to be ready before going to the next step
while [[ $lease_status != "ACTIVE" ]]
do
   sleep 5
   lease_status=$(blazar lease-show --format value -c status "$LEASE_NAME")
done

echo "Lease $LEASE_NAME is ready for business"

Wait until seeing the `Lease NAME_OF_YOUR_LEASE is ready for business` message before continuing. 

Another way is to keep track of PID and wait until it is done.

In any case, expect to receive an email that looks like this shortly:

```
Dear mtavares,


We're sending this email to inform you that your lease mtavares-ovs-test (ID: b8346bd7-1653-40a1-a7f6-c2a2fb7e7939) under project CH-12345678 on CHI@UC will expire on 2019-12-11 13:53:00 UTC / 2019-12-11 07:53:00 Central Time.

You can extend your lease using either the Chameleon web interface or command line interface.


This is an automatic email, please DO NOT reply! If you have any question or issue, please submit a ticket on our help desk.



Thanks,

Chameleon Team
```

Now, some commands can use this lease name but others need a lease ID instead. So, while we are here we might as well get the `$lease_id`.

In [ ]:
lease_id=$(blazar lease-show  --format value -c  reservations "$LEASE_NAME" |grep \"id\"| cut -d \" -f4)

#### Get the ID of the network we will create the baremetal server on
In the previous step we obtained `lease_id`, the ID of the lease named `$USER-default-lease`. Now we will do the same but for the network we will run our server in. Use the `sharednet1` network unless you have a good reason not to such as creating your own networks. Further information on this network is [available in the docs][1].

Creating and using your own networks is beyond the scope of this document.

[1]: https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_basic.html#shared-network

In [ ]:
# Get the network ID associated with sharednet1
network_id=$(openstack network show --format value -c id $PRIVATE_NETWORK_NAME)

### Create a SSH key pair
One of the goals for this document is to access the the baremetal server; that will be achieved by using ssh to connect to the server. For [security][1], servers created in Chameleon are by default accessed using SSH key pair authentication. 

Openstack can store the public key, or keys, which can then be passed to the instance. To see which keys are currently defined you can type

[1]:https://docs.openstack.org/horizon/latest/user/configure-access-and-security-for-instances.html

In [ ]:
openstack keypair list

#### What if I already have a keypair I want to use?

Then skip the next steps and go straight to Get Floating IP.

We will need to create a key pair, say, `$SSHKEY_NAME` in the RSA format with a size of `4096` bits (the minimum size to use nowadays) and saved as `$SSHKEY_FILE` for the private key and `$SSHKEY_FILE.pub` for the public. 

**NOTE:** By default we do not have a `~/.ssh` dir here, so we need to create one first.

In [ ]:
ssh-keygen -t rsa -b 4096  -P '' -C $SSHKEY_NAME -f $SSHKEY_FILE

We are cheating by using `echo "yes"` to say we do not want to use a passphrase associated with this key pair. If you choose to use a passphrase, remove everything before `ssh-keygen`.

Next is to add it to the list of keypairs openstack know to be associated with your account. In reality it is just uploading the public key, which is what you really need to ssh into a host. As with the lease, we do need a [name][2] associated with this key pair:

[2]:https://docs.openstack.org/python-openstackclient/latest/cli/command-objects/keypair.html

In [ ]:
openstack keypair create --public-key $SSHKEY_FILE.pub  $SSHKEY_NAME
openstack keypair list

### Request Floating IP

By default, the server will only have a private IP assigned (in this case, an IP in the `PRIVATE_NETWORK_NAME` network). In order to connect (using SSH or other protocol) to the server from your desktop or another computer in the internet, you should assign a [public-facing floating IP][1]. There are a limited amount of public IPs available across the entire Chameleon testbed, so try to keep the amount of nodes with a public IP to a minimum! A common practice is to set up one node as a "login node" with a public IP, and logging in to that node to manage all of your project's nodes.

[1]: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html#associating-an-ip-address

In [ ]:
# Request a public floating IP (in the 'public' network)
server_ip=$(openstack floating ip create public --format value -c floating_ip_address)
echo "Public IP for this lab is $server_ip"

If the previous step was successful, it will print the public IP, `$server_ip`. Later on we will assign `$server_ip` to the right instance.

#### Should I have more than one floating (public) IP?
The short answer is **no**. Long answer is that 

1. There are very few times when someone needs more than one public IP as opposite to having one instance you can remote in and then go to others. For instance, you could use port forwarding to access all the instances in your experiment directly.
1. There is a finite number if static IPs. By using more than one, someone else may end up with none. This is also the reason why once you finish your lab, you should immediately release the allocated floating IP.

### Create the required baremetal servers

Let's launch a pair of Centos 7 baremetal servers called `$SERVER_NAME-left` and `$SERVER_NAME-right` on network `$PRIVATE_NETWORK_NAME` (identified using `$network_id`) and lease `$LEASE_NAME` (identified using `$lease_id`). Both will be accessible using the ssh keypair `$SSHKEY_NAME` we created earlier.

In [ ]:
for i in left right
do
  openstack server create \
  --flavor "baremetal" \
  --image "CC-CentOS7" \
  --nic net-id="$network_id" \
  --hint reservation="$lease_id" \
  --key-name="$SSHKEY_NAME" \
  --security-group default  \
  "$SERVER_NAME-$i"
done

**NOTE:** In the above loop, we start both instances almost at the same time, without waiting for one to complete. Sometimes you may want to wait on one instance to be fully deployed before continuing; we leave that as an exercise to you.

We could have called the servers `$SERVER_NAME-1`, `$SERVER_NAME-2` and so on or give them more functional names like `$SERVER_NAME-webserver` and `$SERVER_NAME-database`. It all depends on what you want to do with them and how.

#### But wait! And then wait some more!

Creating a server (or node) is not an instantanous process specially if it is a baremetal node. Chameleon has to boot the node, install the OS, move it to the right network, and then it is ready to receive the public IP. All of these steps can take **up to 10 minutes**. If you go back to the previous step you will see a bracket to the left of the line

```bash
for i in left right
```

* If it shows an asterisk (`[*]:`), the step is still running
* If it shows a number (`[15]:`), the step is complete.

Now let's verify that both instances were created properly:

In [ ]:
openstack server list

What we want to see is a status of `ACTIVE` as shown below:

```bash
+--------------------------------------+-----------------------+--------+-------------------------+------------+-----------+
| ID                                   | Name                  | Status | Networks                | Image      | Flavor    |
+--------------------------------------+-----------------------+--------+-------------------------+------------+-----------+
| f1cb353c-705e-4409-8111-08ec6ffe7afe | mtavares-server-right | ACTIVE | sharednet1=10.140.81.84 | CC-CentOS7 | baremetal |
| 84430371-73e6-4c0b-9757-efcc85caf0bd | mtavares-server-left  | ACTIVE | sharednet1=10.140.81.66 | CC-CentOS7 | baremetal |
+--------------------------------------+---------------------------+--------+-------------------------+------------+-----------+
```

for both servers; that indicates they were successfully created and are ready for business. Remember: `BUILD != ACTIVE` and `ERROR != ACTIVE`

Now we need a public floating IP to `$SERVER_NAME-left`. Either you already have a list of IPs you can use with your project or you will [request an IP][2]. For this example we will do the later. You need to know the name of the public network, which in this case is 'public'.

[2]: https://docs.openstack.org/python-openstackclient/latest/cli/command-objects/floating-ip.html#floating-ip-create


In [ ]:
# Assign a public floating IP ONLY to $SERVER_NAME-left
openstack server add floating ip "$SERVER_NAME-left" "$server_ip"

## Access the baremetal server(s)

### Testing the ssh connection
#### `$SERVER_NAME-left`
`$SERVER_NAME-left` now has a publicly facing IP, but can we connect to it?. We will find out using netcat (If you are running this Jupyter book, you have netcat):

In [ ]:
# Check if we can connect to server on port 22.
ssh_status=""
while [ "$ssh_status" != "Up" ]
do
   sleep 30
   ssh_status=$(nc -z "${server_ip}" 22 && echo "Up" || echo "Down")
done

echo "${SERVER_NAME-left} (${server_ip}) is $ssh_status"

The above script will run until it can connect to port 22 (SSH) on `$SERVER_NAME-left` using the public IP. Of course, the answer can be changed so it is is more script-friendly.

But, all that means is that `$SERVER_NAME-left` has ssh up and running and listening on port 22. We need to ssh using the private key to verify it works. The default username for Chameleon-build images is `cc`. While logged in, might as well take a quick look; remember this server will be wiped once we are done.

In [ ]:
login_command="ssh -o \"StrictHostKeyChecking no\" -i $SSHKEY_FILE cc@$server_ip"
eval "$login_command" pwd 

When you see

```bash
/home/cc
``` 

as the output, you validated that you can not only connect to `$SERVER_NAME-left` but also run commands in it.

#### `$SERVER_NAME-right`
`$SERVER_NAME-right` does **not** have a publicly facing IP, so how to connect to it?. We do the netcat step from `$SERVER_NAME-left`

In [ ]:
server_ip_right=$(openstack server list --format value -c Networks --name "$SERVER_NAME-right"| cut -d = -f 2)

eval "$login_command" /bin/bash << EOF
nc -z "${server_ip_right}" 22 && echo "Up" || echo "Down"
EOF

and then the `pwd` test in `$SERVER_NAME-right`

In [ ]:
login_command_right="ssh -o \"StrictHostKeyChecking no\" -i $SSHKEY_FILE -o ProxyCommand=\"ssh -W %h:%p -i $SSHKEY_FILE cc@$server_ip\" cc@$server_ip_right"
eval "$login_command_right" pwd

## Experiment time
### Installing iperf
We need to install [iperf][1] in both servers. So, `$SERVER_NAME-left`

[1]: https://iperf.fr/

In [ ]:
eval "$login_command" sudo yum install -q -y iperf3

and then on `$SERVER_NAME-right`

In [ ]:
eval "$login_command_right" sudo yum install -q -y iperf3

Next, we need to configure iperf such that `$SERVER_NAME-right` is the server and `$SERVER_NAME-left` the client. Since we can't do interactive, we will run iperf in `$SERVER_NAME-right` in the background.

In [ ]:
eval "$login_command_right" -f iperf3 -s

So, let's connect from `$SERVER_NAME-left` and run a simple test

In [ ]:
eval "$login_command" iperf3 -c $server_ip_right 

## Cleaning up after ourselves
As the last task in this document, tear everything down. We can put it all back together by running this jupyter book again later.

1. Delete the public facing IP (so others can use it)

In [ ]:
openstack floating ip delete $server_ip

2. Delete servers. This could be done using a loop.

In [ ]:
openstack server delete $SERVER_NAME-left && \
openstack server delete $SERVER_NAME-right

3. Delete lease

In [ ]:
blazar lease-delete $LEASE_NAME

4. Cleaning the Jupyter book. If you want to bring this book back to its original state, go to *Edit->Clear All Outputs*